In [ ]:
from qiskit import QuantumCircuit
from qiskit_aer import Aer
from qiskit.quantum_info import Statevector
from qiskit.visualization import plot_histogram
import matplotlib.pyplot as plt

# Define the "secret key" to find (3-bit for demonstration)
secret_key = '101'

# Define the oracle that marks the correct key
def create_oracle(secret_key):
    n = len(secret_key)
    oracle = QuantumCircuit(n)
    
    # Apply X-gates where the secret key has '0's
    for i, bit in enumerate(secret_key):
        if bit == '0':
            oracle.x(i)
    
    # Apply multi-controlled Z-gate (as a phase oracle) to mark the correct state
    oracle.h(n - 1)
    oracle.mcx(list(range(n - 1)), n - 1)  # multi-controlled-X (MCX)
    oracle.h(n - 1)
    
    # Reset X-gates where the secret key has '0's
    for i, bit in enumerate(secret_key):
        if bit == '0':
            oracle.x(i)
    
    oracle.name = "Oracle"
    return oracle

# Grover diffusion operator (inversion about the mean)
def create_diffuser(n):
    diffuser = QuantumCircuit(n)
    diffuser.h(range(n))
    diffuser.x(range(n))
    diffuser.h(n - 1)
    diffuser.mcx(list(range(n - 1)), n - 1)
    diffuser.h(n - 1)
    diffuser.x(range(n))
    diffuser.h(range(n))
    diffuser.name = "Diffuser"
    return diffuser

# Construct the Grover's algorithm circuit without measurement
def grover_search(secret_key):
    n = len(secret_key)  # Number of qubits for the key
    grover_circuit = QuantumCircuit(n)
    
    # Step 1: Apply H-gates to create a superposition
    grover_circuit.h(range(n))
    
    # Step 2: Apply Grover's iterations
    oracle = create_oracle(secret_key)
    diffuser = create_diffuser(n)
    
    num_iterations = 1  # Use 1 iteration for simplicity
    for _ in range(num_iterations):
        grover_circuit.compose(oracle, inplace=True)
        grover_circuit.compose(diffuser, inplace=True)
    
    return grover_circuit

# Run the Grover search algorithm without measurement
n_bits = len(secret_key)
grover_circuit = grover_search(secret_key)

# Calculate total gates in the circuit
total_gates = len(grover_circuit.data)

# Use statevector simulation
statevector = Statevector.from_instruction(grover_circuit)
counts = statevector.sample_counts(shots=1024)  # Simulate measurements

# Display results
print("Measurement Results:", counts)
print(f'Total gates used in the Grover circuit: {total_gates}')
plot_histogram(counts)
plt.show()


In [ ]:
import cirq
import numpy as np
from Crypto.Cipher import AES
import os

# Define the AES oracle
def aes_oracle(key: bytes, plaintext: bytes, ciphertext: bytes, qubits) -> list:
    cipher = AES.new(key, AES.MODE_ECB)
    encrypted = cipher.encrypt(plaintext)

    ops = []
    if encrypted == ciphertext:
        for q in qubits:
            ops.append(cirq.X(q))  # Apply X gate as a marker on matching qubits
    return ops


def grover_search(plaintext: bytes, ciphertext: bytes, num_iterations: int):
    num_qubits = 128  # Set to 128 for AES-128 keyspace
    qubits = [cirq.LineQubit(i) for i in range(num_qubits)]

    circuit = cirq.Circuit()
    circuit.append(cirq.H(q) for q in qubits)  # Initialize all 128 qubits in superposition
    key = os.urandom(16)  # Simulated 128-bit AES key for oracle
    circuit.append(aes_oracle(key, plaintext, ciphertext, qubits))  # Oracle with 128-bit input

    # Grover's iterations (for demonstration, using a small number of iterations)
    for _ in range(num_iterations):
        circuit.append(cirq.H(q) for q in qubits)
        circuit.append(cirq.X(q) for q in qubits)
        circuit.append(cirq.Z(qubits[-1])**(len(qubits)-1))
        circuit.append(cirq.X(q) for q in qubits)
        circuit.append(cirq.H(q) for q in qubits)

    circuit.append(cirq.measure(*qubits))  # Measurement on all 128 qubits

    return circuit

# Parameters
plaintext = os.urandom(16)
key = os.urandom(16)
cipher = AES.new(key, AES.MODE_ECB)
ciphertext = cipher.encrypt(plaintext)

# Generate and simulate circuit
num_iterations = 1
circuit = grover_search(plaintext, ciphertext, num_iterations)
simulator = cirq.Simulator()
result = simulator.run(circuit)

# Output results
print("Measured Key:", result)
print("\nQubit Count:", len(circuit.all_qubits()))
print("Total Gate Count:", len(list(circuit.all_operations())))


In [ ]:
import cirq
import numpy as np
from Crypto.Cipher import AES
import os

# Define the Double AES oracle
def double_aes_oracle(key_128: bytes, key_192: bytes, plaintext: bytes, ciphertext: bytes, qubits) -> list:
    # Apply AES-128 encryption
    cipher_128 = AES.new(key_128, AES.MODE_ECB)
    intermediate_ciphertext = cipher_128.encrypt(plaintext)

    # Apply AES-192 encryption
    cipher_192 = AES.new(key_192, AES.MODE_ECB)
    final_ciphertext = cipher_192.encrypt(intermediate_ciphertext)

    # Create a list of operations to mark the solution based on the final ciphertext
    ops = []
    if final_ciphertext == ciphertext:
        for q in qubits:
            ops.append(cirq.X(q))  # Apply X gate as a marker on matching qubits
    return ops


def grover_search_double_aes(plaintext: bytes, ciphertext: bytes, num_iterations: int):
    num_qubits = 128 + 192  # Adjusted qubits for AES-128 + AES-192
    qubits = [cirq.LineQubit(i) for i in range(num_qubits)]

    circuit = cirq.Circuit()

    # Initialize the circuit (Hadamard on all qubits)
    circuit.append(cirq.H(q) for q in qubits)

    # Add the double AES oracle
    key_128 = os.urandom(16)  # Randomly generated AES-128 key for simulation
    key_192 = os.urandom(24)  # Randomly generated AES-192 key for simulation
    circuit.append(double_aes_oracle(key_128, key_192, plaintext, ciphertext, qubits))

    # Apply Grover's diffusion operator
    for _ in range(num_iterations):
        # Inversion about the average
        circuit.append(cirq.H(q) for q in qubits)
        circuit.append(cirq.X(q) for q in qubits)
        circuit.append(cirq.Z(qubits[-1])**(len(qubits)-1))
        circuit.append(cirq.X(q) for q in qubits)
        circuit.append(cirq.H(q) for q in qubits)

    # Measure the result
    circuit.append(cirq.measure(q, key=f'q{i}') for i, q in enumerate(qubits))

    return circuit

# Example parameters
plaintext = os.urandom(16)  # Random 16-byte plaintext
key_128 = os.urandom(16)    # AES-128 key
key_192 = os.urandom(24)    # AES-192 key

# Generate the expected ciphertext for the double AES
cipher_128 = AES.new(key_128, AES.MODE_ECB)
intermediate_ciphertext = cipher_128.encrypt(plaintext)
cipher_192 = AES.new(key_192, AES.MODE_ECB)
ciphertext = cipher_192.encrypt(intermediate_ciphertext)

# Run Grover's search for double AES
num_iterations = 1  # Adjust based on key space size
circuit = grover_search_double_aes(plaintext, ciphertext, num_iterations)
simulator = cirq.Simulator()
result = simulator.run(circuit)

# Output the measurement result
print("Measured Key:", result)
print("\nQubit Count:", len(circuit.all_qubits()))
print("Total Gate Count:", len(list(circuit.all_operations())))


In [ ]:
import cirq
import numpy as np
from Crypto.Cipher import AES
import os

# Define the Double AES oracle
def double_aes_oracle(key_128: bytes, key_192: bytes, plaintext: bytes, ciphertext: bytes, qubits) -> list:
    # Apply AES-128 encryption
    cipher_128 = AES.new(key_128, AES.MODE_ECB)
    intermediate_ciphertext = cipher_128.encrypt(plaintext)

    # Apply AES-192 encryption
    cipher_192 = AES.new(key_192, AES.MODE_ECB)
    final_ciphertext = cipher_192.encrypt(intermediate_ciphertext)

    # Create a list of operations to mark the solution based on the final ciphertext
    ops = []
    if final_ciphertext == ciphertext:
        for q in qubits:
            ops.append(cirq.X(q))  # Apply X gate as a marker on matching qubits
    return ops


def grover_search_double_aes(plaintext: bytes, ciphertext: bytes, num_iterations: int):
    num_qubits = 128 + 192  # Adjusted qubits for AES-128 + AES-192
    qubits = [cirq.LineQubit(i) for i in range(num_qubits)]

    circuit = cirq.Circuit()

    # Initialize the circuit (Hadamard on all qubits)
    circuit.append(cirq.H(q) for q in qubits)

    # Add the double AES oracle
    key_128 = os.urandom(16)  # Randomly generated AES-128 key for simulation
    key_192 = os.urandom(24)  # Randomly generated AES-192 key for simulation
    circuit.append(double_aes_oracle(key_128, key_192, plaintext, ciphertext, qubits))

    # Apply Grover's diffusion operator
    for _ in range(num_iterations):
        # Inversion about the average
        circuit.append(cirq.H(q) for q in qubits)
        circuit.append(cirq.X(q) for q in qubits)
        circuit.append(cirq.Z(qubits[-1])**(len(qubits)-1))
        circuit.append(cirq.X(q) for q in qubits)
        circuit.append(cirq.H(q) for q in qubits)

    # Measure the result
    circuit.append(cirq.measure(q, key=f'q{i}') for i, q in enumerate(qubits))

    return circuit

# Example parameters
plaintext = os.urandom(16)  # Random 16-byte plaintext
key_128 = os.urandom(16)    # AES-128 key
key_192 = os.urandom(24)    # AES-192 key

# Generate the expected ciphertext for the double AES
cipher_128 = AES.new(key_128, AES.MODE_ECB)
intermediate_ciphertext = cipher_128.encrypt(plaintext)
cipher_192 = AES.new(key_192, AES.MODE_ECB)
ciphertext = cipher_192.encrypt(intermediate_ciphertext)

# Run Grover's search for double AES
num_iterations = 1  # Adjust based on key space size
circuit = grover_search_double_aes(plaintext, ciphertext, num_iterations)
simulator = cirq.Simulator()
result = simulator.run(circuit)

# Output the measurement result
print("Measured Key:", result)
print("\nQubit Count:", len(circuit.all_qubits()))
print("Total Gate Count:", len(list(circuit.all_operations())))
